## Installing & importing packages

In [0]:
%tensorflow_version 1.x

In [91]:
!pip install tflearn

In [0]:
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

In [0]:
import numpy as np
import tensorflow as tf
import tflearn
import random
import json
import pickle
import re

## Data parsing - cleaning & One Hot Encoding



In [0]:
with open('/content/drive/My Drive/Colab Notebooks/github/deep-learning-chatbot/data/intents.json') as file:
  data = json.load(file)

In [95]:
nltk.download('punkt')
try:
    with open("/content/drive/My Drive/Colab Notebooks/github/deep-learning-chatbot/data/data.pickle", "rb") as f:
        words, labels, training, output = pickle.load(f)
        print('Pickle Loaded...')
except:
    words = []
    labels = []
    docs_x = []
    docs_y = []

    for intent in data["intents"]:
        for pattern in intent["patterns"]:
            wrds = nltk.word_tokenize(pattern)
            words.extend(wrds)
            docs_x.append(wrds)
            docs_y.append(intent["tag"])

        if intent["tag"] not in labels:
            labels.append(intent["tag"])

    words = [stemmer.stem(w.lower()) for w in words if w != "?"]
    words = sorted(list(set(words)))

    labels = sorted(labels)

    training = []
    output = []

    output_empty = [0 for _ in range(len(labels))]

    for x, doc in enumerate(docs_x):
        bag = []

        wrds = [stemmer.stem(w.lower()) for w in doc]

        for w in words:
            if w in wrds:
                bag.append(1)
            else:
                bag.append(0)

        output_row = output_empty[:]
        output_row[labels.index(docs_y[x])] = 1

        training.append(bag)
        output.append(output_row)


    training = np.array(training)
    output = np.array(output)

    with open("data.pickle", "wb") as f:
        pickle.dump((words, labels, training, output), f)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Pickle Loaded...


## Training Model

In [96]:
tf.reset_default_graph()

net = tflearn.input_data(shape=[None, len(training[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(output[0]), activation="softmax")
net = tflearn.regression(net)

model = tflearn.DNN(net)
# model.fit(training, output, n_epoch=10000, batch_size=8, show_metric=True)
# model.save("model.tflearn")
try:
    model.load("/content/drive/My Drive/Colab Notebooks/github/deep-learning-chatbot/training/model.tflearn")
    print('Model Loaded....')
except:
    model.fit(training, output, n_epoch=1000, batch_size=8, show_metric=True)
    model.save("model.tflearn")

INFO:tensorflow:Restoring parameters from /content/drive/My Drive/Colab Notebooks/github/deep-learning-chatbot/training/model.tflearn
Model Loaded....


## Bot 

In [0]:
def check_bargain_module(response):
  response_chunks = re.sub("[^#\w]", " ",  response).split()

  for chunk in response_chunks:
    if chunk.startswith('#PD'):
      with open('/content/drive/My Drive/Colab Notebooks/github/deep-learning-chatbot/data/products.json') as f:
        products = json.load(f)

      if chunk in products:
        product = products[chunk]
        return [True, product]
      else:
        print("BOT: No Product found! make sure you double checked the product code.")
        return [False, None]
  return [False, "CNN"]
      
      

In [0]:
def bag_of_words(s, words):
    bag = [0 for _ in range(len(words))]

    s_words = nltk.word_tokenize(s)
    s_words = [stemmer.stem(word.lower()) for word in s_words]

    for se in s_words:
        for i, w in enumerate(words):
            if w == se:
                bag[i] = 1
            
    return np.array(bag)

In [0]:
def bargain_response(product, meta_score):
  product_name = product['name']
  claim_score = product['claim_score']
  cost_price = product['cost_price']
  store_price = cost_price + ( cost_price * product['exp_profit_percent'] / 100 )

  profit_theshold = calculate_profit_threshold(claim_score, meta_score)

  offered_price = cost_price + ( cost_price * profit_theshold / 100 )

  print(f'Product name: {product_name} \nPrice: {store_price}')
  print('I can offer you the discounted price at:', str(round(offered_price, 2)))

In [0]:
def cnn_response(inp):
    threshold = 0.50
    results = model.predict([bag_of_words(inp, words)])
    results_index = np.argmax(results)
    tag = labels[results_index]

    if results[0, results_index] > threshold:
      for tg in data["intents"]:
          if tg['tag'] == tag:
              responses = tg['responses']

      print(f'Bot: {random.choice(responses)}')
    else:
      print("Bot: Sorry, I don't quite understand. Can you please elaborate please?")

In [0]:
def chat():
    USER_META_SCORE = 8.0
    print('Now you are talking with Bargainly\n(type "--quit" to stop)')
    while True:
      inp = str(input("You: "))
      if inp.lower() == "--quit":
          break
      checker = check_bargain_module(inp)
      is_bargain_module = checker[0]
      if is_bargain_module:
        bargain_response(checker[1], USER_META_SCORE)
      elif checker[1] == 'CNN':
        cnn_response(inp)

In [87]:
!pip install -U scikit-fuzzy
import numpy as np
import skfuzzy as fuzz
from skfuzzy import control as ctrl

Requirement already up-to-date: scikit-fuzzy in /usr/local/lib/python3.6/dist-packages (0.4.2)


In [0]:
def calculate_profit_threshold(claim_score, meta_score):
  product_claim_score = ctrl.Antecedent(np.arange(0, 11, 1), 'product_claim_score')
  user_meta_score = ctrl.Antecedent(np.arange(0, 11, 1), 'user_meta_score')
  profit_threshold_percent = ctrl.Consequent(np.arange(0, 26, 1), 'profit_threshold_percent')

  # Product Claim Score membership function
  product_claim_score['low'] = fuzz.trimf(product_claim_score.universe, [0, 0, 5])
  product_claim_score['medium'] = fuzz.trimf(product_claim_score.universe, [0, 5, 10])
  product_claim_score['high'] = fuzz.trimf(product_claim_score.universe, [5, 10, 10])

  # User Meta Score membership function
  user_meta_score['low'] = fuzz.trimf(user_meta_score.universe, [0, 0, 5])
  user_meta_score['medium'] = fuzz.trimf(user_meta_score.universe, [0, 5, 10])
  user_meta_score['high'] = fuzz.trimf(user_meta_score.universe, [5, 10, 10])

  # Profit Thereshold membership function
  profit_threshold_percent['low'] = fuzz.trimf(profit_threshold_percent.universe, [0, 0, 13])
  profit_threshold_percent['medium'] = fuzz.trimf(profit_threshold_percent.universe, [0, 13, 25])
  profit_threshold_percent['high'] = fuzz.trimf(profit_threshold_percent.universe, [13, 25, 25])

  # Fuzzy rules
  rule1 = ctrl.Rule(product_claim_score['low'] | user_meta_score['high'], profit_threshold_percent['low'])
  rule2 = ctrl.Rule(product_claim_score['medium'] |user_meta_score['medium'], profit_threshold_percent['medium'])
  rule3 = ctrl.Rule(product_claim_score['high'] | user_meta_score['low'], profit_threshold_percent['high'])

  profit_threshold_percent_ctrl = ctrl.ControlSystem([rule1, rule2, rule3])

  profit_thresholding = ctrl.ControlSystemSimulation(profit_threshold_percent_ctrl)

  profit_thresholding.input['product_claim_score'] = claim_score
  profit_thresholding.input['user_meta_score'] = meta_score

  # Computing result
  profit_thresholding.compute()

  return profit_thresholding.output['profit_threshold_percent']

In [125]:
calculate_profit_threshold(claim_score = 8.5, meta_score = 1)

15.856086580086581

## Let's chat with the bot

In [0]:
chat()